Make proper comments as the comments are not correct.


In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import scipy as sp
from numpy import linalg as LA
import networkx.linalg.algebraicconnectivity as alg
import collections 
import math
import pandas as pd


Results_ncut = pd.DataFrame(index = ['kannan', 'UnL' , 'SymL', 'RwL'])
Results_conductance = pd.DataFrame(index = ['kannan', 'UnL' , 'SymL', 'RwL'])
Graphs = []

In [3]:

# This wil get us the similarity matrix W given the graph and the number of vertices n.
# the vertices are number form 0 to n - 1.
def get_matrix1(graph, n):   
    
    w = np.zeros((n, n))

    for i in range(0, n):
        for j in range(0, n):
            if(i != j):
                w[i][j] =  1/((graph[i,0] - graph[j,0])**2 + (graph[i, 1] - graph[j, 1])**2)**(1/2)
    print(w)
    return w


In [4]:

# This wil get us the similarity matrix W given the graph and the number of vertices n.
# the vertices are number form 0 to n - 1.
def get_matrix2(graph, n):
    
    
    w = np.zeros((n, n))
   
    for x in graph.keys() :
        for y in graph[x]:
            w[x][y] = 1
            w[y][x] = 1
    return w
    

In [5]:
# returns the fielder eigen vector given the similartiy matrix W.

def get_fielder(b, n):  #Similairty matix W

    eigvals, eigvecs =sp.linalg.eigh(b)# eigen values and eigen vectors of the unnormalized laplacian D - W 
    #print(eigvals, eigvecs)
    fielder = eigvecs.T[-2, :] #The 2nd largest eigen vector of the laplacian L = D - W
    
    return fielder



In [6]:
# Here we use our current algorithm to partition the graph
# This returns the clusters, the cluster_name vector and the edeges cut
def get_clusters(f, w, n): # the similarity matrix and the number of vertices
    
    threshold_index = 0
    index = np.argsort(f)
    #print(index)
    
    best_conductance = 2
    cluster_name = [0]*n
    clusters = [[], []]
    
    for i in range(0, n - 1):
        
        cluster_name[index[i]] = 1
        nc, c = ncut(cluster_name, w, n)
        #print(i, nc, c, best_conductance)
        if(c <= best_conductance):
            best_conductance = c
            threshold_index = i
    
    cluster_name = [0]*n
    
    for i in range(0,threshold_index + 1):
        cluster_name[index[i]] = 1
    
    for i in range(0, n):
        clusters[cluster_name[i]].append(i)
        
    # this part changes the cluster_name from a list to numpy array (This step helps to write easy codes)
    cluster_name = np.asarray(cluster_name)
    cluster_name = np.reshape(cluster_name, (1,n))
    
    cp = cluster_name.flatten().tolist()
    
    
    return clusters, cluster_name


In [7]:
def ncut(cluster_name, w, n):
    
    cluster_name = np.asarray(cluster_name)
    cluster_name = np.reshape(cluster_name, (1,n))
    
    mull_1 = cluster_name
    mull_2 = np.ones((1,n)) - mull_1
    
    w_1 = np.dot(mull_1, np.dot(w, np.ones((n,1))))
    w_2 = np.dot(mull_2, np.dot(w, np.ones((n,1))))
    
    
    cut = np.dot(mull_1, np.dot(w,mull_2.T))
    #print(cut, w_1,w_2)
    
    ans = cut*(1/w_1 + 1/w_2)
    ans_1 = cut/min(w_1, w_2)
    
    return ans[0][0], ans_1[0][0] 

    

In [8]:
def get_cut(clusters, w, n):
    cut = []
    for x in clusters[0]:
        for y in clusters[1]:
            if(w[x][y] != 0):
                cut.append([x,y])
    return cut

In [9]:
def get_results(b, n, w):
    
    fielder = get_fielder(b, n)
    #print(fielder)
    add_col_ncut = []
    add_col_con = []
    
    clusters, cluster_predict = get_clusters(fielder, w, n) 
    nc, c = ncut(cluster_predict, w, n)
    print("\nCurrent Algorithm\n")
    print("Ncut value:", nc)
    print("Conductance:", c)
    
    add_col_ncut.append(nc)
    add_col_con.append(c)
    
    return add_col_ncut, add_col_con

In [10]:
def get_results1(b, n, w):
    
    
    fielder = get_fielder(b, n)
    add_col_ncut = []
    add_col_con = []
    
    clusters, cluster_predict = get_clusters(fielder, w, n) 
    nc, c = ncut(cluster_predict, w, n)
    
    add_col_ncut.append(nc)
    add_col_con.append(c)
    
    
    return add_col_ncut, add_col_con

In [11]:
def show_results(graph, print_results):
    
    name = graph[0]
    graph = graph[1]
    n = graph[1][0]
    graph = graph[0][0]
    w = get_matrix2(graph, n)
    D = w.sum(axis = 1)
    D = np.reciprocal(D)
    D = np.diag(D)
    b = np.dot(D, w)
    
    
    if(print_results == 1):
        add_col_ncut, add_col_con = get_results(b, n, w)
    else:
        add_col_ncut, add_col_con = get_results1(b, n, w)
    
    Results_ncut[name], Results_conductance[name] = add_col_ncut, add_col_con
    return 0

In [12]:
def show_results1(graph, print_results):
    
   
    name = graph[0]
    graph = graph[1]
    n = graph[1][0]
    w = graph[0]
    print(n)
    D = w.sum(axis = 1)
    D = np.reciprocal(D)
    D = np.diag(D)
    b = np.dot(D, w)
    
    
    if(print_results == 1):
        add_col_ncut, add_col_con = get_results(b, n, w)
    else:
        add_col_ncut, add_col_con = get_results1(b, n, w)
    
    Results_ncut[name], Results_conductance[name] = add_col_ncut, add_col_con
    return 0

In [13]:
Petersen = ['Petersen', [[{0 : [2, 3],
         1 : [4, 3],
         2 : [0, 4],
         3 : [0, 1],
         4 : [1, 2],
         5 : [6, 0],
         6 : [7, 1],
         7 : [8, 2],
         8 : [9, 3],
         9 : [5, 4]
             }],
        [10]]]

Graphs.append(Petersen)
show_results(Petersen, True)


Current Algorithm

Ncut value: 0.9333333333333333
Conductance: 0.4666666666666667


0

In [14]:
Enneahedron = ['Enneahedron', [[{0 : [1, 3, 4],
         1 : [0, 2, 4, 5],
         2 : [1, 3, 6],
         3 : [0, 7, 6, 2],
         4 : [0, 1, 5, 6, 7],
         5 : [1, 4, 6],
         6 : [2, 3, 4, 5, 7],
         7 : [4, 3, 6],
             }],
        [8]]]
Graphs.append(Enneahedron)
show_results(Enneahedron, True)


Current Algorithm

Ncut value: 0.6666666666666666
Conductance: 0.3333333333333333


0

In [15]:
Enneahedron_1 = ['Enneahedron_1',[[{0 : [1, 3, 4],
         1 : [0, 2, 4, 6, 5],
         2 : [1, 3, 5],
         3 : [0, 4, 7, 5, 2],
         4 : [0, 1, 3, 6],
         5 : [1, 7, 3, 2],
         6 : [1, 3, 4, 7],
         7 : [6, 5],
             }],
        [8]]]
Graphs.append(Enneahedron_1)
show_results(Enneahedron_1, True)


Current Algorithm

Ncut value: 0.8784313725490196
Conductance: 0.4666666666666667


0

In [16]:
Cockroach = ['Cockroach',[[{0 : [1],
         1 : [0, 2],
         2 : [1, 3],
         3 : [2, 4, 8],
         4 : [3, 5, 7],
         5 : [4, 6],
         6 : [5, 7],
         7 : [6, 8, 4],
         8 : [7, 9, 3],
         9 : [8, 10],
         10 : [11]
             }],
        [12]]]
Graphs.append(Cockroach)
show_results(Cockroach, True)


Current Algorithm

Ncut value: 0.24761904761904763
Conductance: 0.2


0

In [17]:
Cockroach_1 = ['Cockroach_1', [[{0 : [1, 11],
         1 : [0, 2],
         2 : [1, 3],
         3 : [2, 4, 8],
         4 : [3, 5, 7],
         5 : [4, 6],
         6 : [5, 7],
         7 : [6, 8, 4],
         8 : [7, 9, 3],
         9 : [8, 10],
         10 : [11],
         11 : [0]    }],
        [12]]]

Graphs.append(Cockroach_1)
show_results(Cockroach_1, True)


Current Algorithm

Ncut value: 0.29166666666666663
Conductance: 0.16666666666666666


0

In [18]:
Path_point  = ['Path_point',[[{0 : [1],
         1 : [0, 2],
         2 : [1, 3],
         3 : [2, 4],
         4 : [3, 5],
         5 : [4, 6],
         6 : [5, 7],
         7 : [6, 8],
         8: [7, 9],
         9: [8, 10],
         10:[9, 11],
         11 :[0,1,2,3,4,5,6,7,8,9,10]
                }],
        [12]]]

Graphs.append(Path_point)
show_results(Path_point, True)


Current Algorithm

Ncut value: 0.6917647058823528
Conductance: 0.4117647058823529


0

In [19]:
Two_clusters = ['Two_clusters', [[{0 : [1, 2, 3],
         1 : [0, 2, 3, 4],
         2 : [0, 1, 3],
         3 : [0, 1, 2],
         4 : [5, 6, 7],
         5 : [4, 6, 7],
         6 : [4, 5, 7],
         7 : [4, 5, 6]
        }],
        [8]]]

Graphs.append(Two_clusters)
show_results(Two_clusters, True)


Current Algorithm

Ncut value: 0.15384615384615385
Conductance: 0.07692307692307693


0

In [20]:
Discrete_moons = {}

for i in range(0, 19):
    Discrete_moons[i] = [i + 1]
    Discrete_moons[i + 20] = [i + 21]

for i in range(0, 10):
    Discrete_moons[i].append(19 - i)
    Discrete_moons[i + 20].append(39 - i)

for i in range(0, 5):
    Discrete_moons[i + 10].append(24 - i)
#print(Discrete_moons)

D_M = ['Discrete_moons',[[Discrete_moons], [40]]]

Graphs.append(D_M)
show_results(D_M, True)


Current Algorithm

Ncut value: 0.09270516717325228
Conductance: 0.07142857142857142


0

In [21]:
Double_Tree = {}

for i in range(1,10):
    Double_Tree[i] = [math.floor((i - 1)/2)]
    Double_Tree[i + 10] = [10 + math.floor((i - 1)/2)]

Double_Tree[0] = [10]

D_T = ['Double_Tree',[[Double_Tree], [20]]]

Graphs.append(D_T)
show_results(D_T, True)


Current Algorithm

Ncut value: 0.10526315789473684
Conductance: 0.05263157894736842


0

In [22]:
Double_Tree1 = {}

for i in range(1,10):
    Double_Tree1[i] = [math.floor((i-1)/2)]
    Double_Tree1[i + 10] = [10 + math.floor((i-1)/2)]

Double_Tree1[0] = [10]

for i in range(5,9):
     Double_Tree1[i].append(10+i)
for i in range(0,9):
    Double_Tree1[i].append((i+1)%10)
D_T1 = ['Double_Tree1',[[Double_Tree1], [20]]]

Graphs.append(D_T1)
show_results(D_T1, True)




Current Algorithm

Ncut value: 0.5166666666666666
Conductance: 0.26666666666666666


0

In [23]:
for graph in Graphs:
    show_results(graph, False)
    
    

In [24]:
print(Results_conductance.T)


                  kannan
Petersen        0.466667
Enneahedron     0.333333
Enneahedron_1   0.466667
Cockroach       0.200000
Cockroach_1     0.166667
Path_point      0.411765
Two_clusters    0.076923
Discrete_moons  0.071429
Double_Tree     0.052632
Double_Tree1    0.266667
